In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
from tqdm import tqdm

In [2]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 500
DATA_NAME = "auto0"
TOPO_NAME = "4"

task = pd.read_csv("../../data/stream/stream_38_32.csv")
network = pd.read_csv("../../data/stream/stream_topology.csv")

for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

In [3]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [4]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['t_proc'] = int(row['t_proc'])
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1
g = nx.from_numpy_matrix(net)

## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

In [5]:
class Node:
    def __init__(self, i, f, pi, ci, di, path, offset):
        self.i = i
        self.f = f
        self.pi = pi
        self.ci = ci
        self.di = di
        self.path = path
        self.offset = offset
        self.init()
        
    
    def init(self):
        self.r = np.zeros(len(index_to_link))
        self.t = np.zeros(len(index_to_link))
        for index, link in enumerate(self.path):
            self.r[link_to_index[link]] = 1
            if index == 0:
                self.t[link_to_index[link]] = self.offset
            else:
                self.t[link_to_index[link]] = self.t[link_to_index[self.path[index - 1]]] + self.ci + net_var[link]['t_proc']


In [6]:
def conflict(i: Node, j: Node):
    global net_var, index_to_link
    r_i = np.nonzero(i.r)[0]
    r_j = np.nonzero(j.r)[0]

    if i.i == j.i:
        return False
    if not set(r_i) & set(r_j):
        return False

    for link in set(r_i) & set(r_j):
        lcm = np.lcm(i.pi, j.pi)
        for a, b in [
                        (a, b) for a in range(0, int(lcm / i.pi))
                            for b in range(0, int(lcm / j.pi))
                    ]:
            jgi = j.t[link] + b * j.pi >= i.t[link] + i.ci + a * i.pi
            igj = i.t[link] + a * i.pi >= j.t[link] + j.ci + b * j.pi
            if not (jgi or igj):
                return True
    return False

# a = Node(0, 1, 100, 10, 100, ['(0, 1)', '(1, 2)'], 11)
# b = Node(1, 3, 20, 10, 20, ['(0, 1)', '(1, 2)'], 0)
# print(conflict(a,b))


In [7]:
def get_paths(g, start, end):
    paths = nx.all_simple_paths(g, start, end)
    return [[str((v, path[h + 1]))
            for h, v in enumerate(path[:-1])] for path in paths]

In [8]:
# phases = [list(range(0, task.loc[i]['period'] - task.loc[i]['size'] * 8)) for i in range(len(task))]
# paths = [get_paths(g, task.loc[i]['src'], eval(task.loc[i]['dst'])[0]) for i in range(len(task))]
# current_state = [[0, 0] for i in range(len(task))] ## [phase, path]

In [9]:
def stateful_generator(i):
    ## Not reach the end of the paths
    global current_state, phases, paths
    if current_state[i][1] < len(paths[i]) - 1:
        current_state[i][1] += 1
        return (phases[i][current_state[i][0]], paths[i][current_state[i][1]])
    elif current_state[i][0] < len(phases[i]) - 1:
        current_state[i][0] += 1
        current_state[i][1] = 0
        return (phases[i][current_state[i][0]], paths[i][current_state[i][1]])
    else:
        return None

In [10]:
def add_vertex(v: Node):
    global CG
    CG.add_node(v.i, config = v)
    for node in CG.nodes:
        if node == v.i:
            continue
        if conflict(v, CG.nodes[node]['config']):
            CG.add_edge(v.i, node)
    

In [11]:
def generator(task_set):
    global task, num_node
    flag = False
    for i in task_set:
        phase, path = stateful_generator(i)
        config = Node(num_node, i, task.loc[i]['period'], task.loc[i]['size'] * 8, task.loc[i]['deadline'], path, phase)
        add_vertex(config)
        num_node += 1
        flag = True
    return flag

In [12]:
# CG = nx.Graph()

In [13]:
# add_vertex(a)
# print(CG)
# add_vertex(b)
# print(CG)
# c = Node(2, 5, 20, 10, 20, ['(0, 1)', '(1, 2)'], 20)
# print(conflict(a, c))
# print(conflict(b, c))
# add_vertex(c)
# print(CG)

In [14]:
# sc = np.zeros(len(task)) + 1

In [15]:
def luby():
    global CG, sc
    a = 0.7

    CG_copy = CG.copy()

    I = set()
    while CG_copy.nodes:
        X = set()
        for node in [x for x in CG_copy.nodes]:
            if nx.degree(CG_copy, node) == 0:
                I.add(node)
                CG_copy.remove_node(node)
                continue
            p_deg = 1 / (2 * nx.degree(CG_copy, node))
            p_sc = 1 - ((sc[CG.nodes[node]['config'].f] + 1) / (max(sc) + 1))
            if np.random.random() < a * p_deg + (1 - a) * p_sc:
                X.add(node)
                
        I_p = X
        edges = list(CG_copy.subgraph(I_p).edges)
        while edges:
            link = edges.pop()
            if nx.degree(CG_copy, link[0]) <= nx.degree(CG_copy, link[1]):
                I_p.remove(link[0])
            else:
                I_p.remove(link[1])
            edges = list(CG_copy.subgraph(I_p).edges)

        # for link in CG_copy.subgraph(I_p).edges:
        #     if nx.degree(CG_copy, link[0]) <= nx.degree(CG_copy, link[1]):
        #         I_p.remove(link[0])
        #     else:
        #         I_p.remove(link[1])
        I = I | I_p
        Y = I_p | set().union(*(CG_copy.neighbors(n) for n in I_p))
        CG_copy.remove_nodes_from(Y)
    return I

# GC = nx.Graph()
# add_vertex(Node(0, 0, 100, 10, 20, ['(0, 1)', '(1, 2)'], 0))
# add_vertex(Node(1, 1, 100, 10, 100, ['(0, 1)', '(1, 2)'],10 - 1))
# add_vertex(Node(2, 2, 100, 10, 20, ['(0, 1)', '(1, 2)'], 20 - 2))
# add_vertex(Node(3, 3, 100, 10, 20, ['(0, 1)', '(1, 2)'], 30 - 3))
# add_vertex(Node(4, 4, 100, 10, 20, ['(0, 1)', '(1, 2)'], 40 - 4))
# add_vertex(Node(5, 5, 100, 10, 20, ['(0, 1)', '(1, 2)'], 50 - 5))
# add_vertex(Node(6, 6, 100, 10, 20, ['(0, 1)', '(1, 2)'], 60 - 6))
# nx.draw(CG, with_labels=True)
# independent_set = luby()
# print(independent_set)
    

In [16]:
def ILP():
    global CG
    CG_copy = CG.copy()
    m = gp.Model('ILP')
    m.setParam('OutputFlag', False)
    m.setParam('timeLimit', 5 * 60)
    xv = m.addVars(CG_copy.nodes, vtype=GRB.BINARY, name='x')
    xs = m.addVars(len(task), vtype=GRB.BINARY, name='s')
    m.setObjective(xs.sum(), GRB.MAXIMIZE)
    for edge in CG_copy.edges:
        m.addConstr(xv[edge[0]] + xv[edge[1]] <= 1)
    for i in range(len(task)):
        m.addConstr(xs[i] <= sum(xv[k] for k in CG_copy.nodes if CG.nodes[k]['config'].f == i))
    m.optimize()
    return set([k for k in CG_copy.nodes if xv[k].x == 1])


In [17]:
def nods_to_streams(nodes):
    global CG
    streams = set()
    for node in nodes:
        streams.add(CG.nodes[node]['config'].f)
    return streams

In [18]:
# ILP_set = ILP()
# print(ILP_set)

In [19]:
# num_covered = []
# opt_window_size = 10

def trigger_sure():
    global num_covered, opt_window_size
    opt_window_size = min(opt_window_size, 20)
    window = num_covered[-opt_window_size:]
    d_past = np.sum(np.diff(window))
    if d_past > 0:
        opt_window_size += 1
        return False
    else:
        opt_window_size = 10
        return True
    

In [20]:
# p_thres = 1
# graph_window_size = 5
def trigger_completion(miss_num):
    global num_covered, graph_window_size, p_thres
    graph_window_size = min(graph_window_size, 20)
    window_old = num_covered[-(graph_window_size + 1): -1]
    window_new = num_covered[-graph_window_size:]
    if not window_old:
        n_old = 0
    else:
        n_old = round(np.mean(window_old))
    n_new = round(np.mean(window_new))
    if n_new < n_old:
        p_thres -= 1
        graph_window_size -= 1
    elif n_new > n_old:
        p_thres += 1
        graph_window_size += 1
    else:
        p_thres -= 1
    if miss_num > p_thres:
        return True
    else:
        return False


In [21]:
## Global variables to remember searching state
num_node = 0
phases = [list(range(0, task.loc[i]['period'] - task.loc[i]['size'] * 8)) for i in range(len(task))]
paths = [get_paths(g, task.loc[i]['src'], eval(task.loc[i]['dst'])[0]) for i in range(len(task))]

for i in range(len(task)):
    paths[i] = [path for path in paths[i] if len(path) * (task.loc[i]['size'] * 8 + 10) <= task.loc[i]['deadline']]

for x in paths:
    if len(x) == 0:
        print('No path available')
        raise Exception

for i in range(len(task)):
    np.random.shuffle(paths[i])
    np.random.shuffle(phases[i])

current_state = [[0, 0] for i in range(len(task))] ## [phase, path]


## Global: conflict graph
CG = nx.Graph()
## Global: number of covered tasks of heuristic
sc = np.zeros(len(task))
## Historical number of covered tasks
num_covered = []
## If the missing streams large than p_thres, then trigger generate function
p_thres = len(task) // 2
opt_window_size = 5
graph_window_size = 5



In [22]:
### Algorithm

flag = True
while flag:
    flag = generator(list(range(len(task))))
    I = luby()
    covered_streams = nods_to_streams(I)
    missed_streams = set(range(len(task))) - covered_streams
    num_covered.append(len(covered_streams))
    sc[list(covered_streams)] += 1
    print('Luby Triggered: | graph size-%d | IMS size-%d | covered-%d |'%(len(CG.nodes), len(I), len(covered_streams)))
    # print(missed_streams)
    if trigger_completion(len(missed_streams)):
        generator(missed_streams)
    if trigger_sure():
        I = ILP()
        coverd_streams = nods_to_streams(I)
        missed_streams = set(range(len(task))) - covered_streams
        num_covered.append(len(covered_streams))
        sc[list(covered_streams)] += 1
        print('ILP  Triggered: | graph size-%d | IMS size-%d | covered-%d |'%(len(CG.nodes), len(I), len(covered_streams)))
        # print(missed_streams)
        if trigger_completion(len(missed_streams)):
            generator(missed_streams)
    if len(covered_streams) == len(task):
        print('SOLVED')
        break
else:
    print('NOT SOLVED')


Luby Triggered: | graph size-38 | IMS size-29 | covered-29 |
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
ILP  Triggered: | graph size-38 | IMS size-30 | covered-29 |
Luby Triggered: | graph size-76 | IMS size-33 | covered-33 |
Luby Triggered: | graph size-114 | IMS size-34 | covered-34 |
Luby Triggered: | graph size-152 | IMS size-41 | covered-36 |
Luby Triggered: | graph size-190 | IMS size-45 | covered-31 |
Luby Triggered: | graph size-228 | IMS size-56 | covered-36 |
Luby Triggered: | graph size-266 | IMS size-53 | covered-35 |
Luby Triggered: | graph size-304 | IMS size-61 | covered-36 |
Luby Triggered: | graph size-342 | IMS size-70 | covered-37 |
Luby Triggered: | graph size-380 | IMS size-71 | covered-36 |
Luby Triggered: | graph size-418 | IMS size-72 | covered-36 |
Luby Triggered: | graph size-456 | IMS size-76 | covered-37 |
Luby Triggered: | graph size-494 | IMS size-82 | covered-38 |
SOLVED


In [23]:
for i in range(len(task)):
    print(CG.nodes[i]['config'].path, CG.nodes[i]['config'].offset)

['(13, 5)', '(5, 4)', '(4, 3)', '(3, 2)', '(2, 1)', '(1, 0)', '(0, 8)'] 1820
['(11, 3)', '(3, 2)', '(2, 1)', '(1, 0)', '(0, 7)', '(7, 6)', '(6, 14)'] 1167
['(12, 4)', '(4, 5)', '(5, 6)', '(6, 7)', '(7, 0)', '(0, 1)', '(1, 2)', '(2, 10)'] 1783
['(10, 2)', '(2, 1)', '(1, 6)', '(6, 14)'] 3763
['(9, 1)', '(1, 2)', '(2, 5)', '(5, 4)', '(4, 3)', '(3, 11)'] 3524
['(10, 2)', '(2, 1)', '(1, 0)', '(0, 7)', '(7, 6)', '(6, 5)', '(5, 4)', '(4, 12)'] 3863
['(12, 4)', '(4, 5)', '(5, 6)', '(6, 1)', '(1, 0)', '(0, 8)'] 977
['(13, 5)', '(5, 2)', '(2, 3)', '(3, 11)'] 1532
['(10, 2)', '(2, 5)', '(5, 6)', '(6, 7)', '(7, 0)', '(0, 1)', '(1, 9)'] 568
['(12, 4)', '(4, 5)', '(5, 13)'] 3260
['(8, 0)', '(0, 7)', '(7, 6)', '(6, 14)'] 3310
['(11, 3)', '(3, 4)', '(4, 5)', '(5, 6)', '(6, 7)', '(7, 0)', '(0, 1)', '(1, 9)'] 3396
['(12, 4)', '(4, 5)', '(5, 6)', '(6, 1)', '(1, 0)', '(0, 7)', '(7, 15)'] 513
['(11, 3)', '(3, 2)', '(2, 1)', '(1, 0)', '(0, 7)', '(7, 6)', '(6, 5)', '(5, 4)', '(4, 12)'] 3010
['(9, 1)', '(1, 6

: 

: 

: 

: 

...

: 

: 

: 

: 

: 

In [ ]:
window = num_covered[-opt_window_size:]
np.sum(np.diff(window))

: 